## TUGAS 4 (Pipelines)

Pertama tama saya memisahkan kolom Petal Length, Petal Width, Sepal Length, dan Sepal Width + class dan ditambah masing-masing kolom id di csv. kemudian saya mengkonversi csv to sql menjadi script sql. setelah masing-masing kolom telah di ubah ke script sql, selanjutnya saya memasukkan data tersebut ke masing” aplikasi database yaitu mysql, sql server, dan postgres sql local + elephant. Pertama, kolom sepal length saya masukkan ke database postgressql local, kemudian kolom sepal width saya masukkkan ke database postgressql elephantsql, kolom petal length dimasukkan ke database mysql local, dan terakhir kolom petalwidth +class dimasukkan ke database sql server, semua dimasukkkan sesuai dengan tahapan masing” database tersebut dengan menggunakan script sql yang sudah dikonversi dari csv. kemudian setelah berhasil memasukkan ke semua database lanjut menjadikan 1 kolom-kolom tersebut. disini saya mencoba menggunakan orchest namun tidak bisa untuk yang local (hanya bisa postgrest elephantsql saja) sehingga saya mencoba menggabungkan data-data dari tiap database tersebut dengan menggunakan google colab.

http://localhost:8888/?token=ea4f8fda93fa3a5e19da8685b1711e55e4d4db20098d712e

### Mengambil data dari database SQL Server

In [3]:
pip install pyodbc 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 18.5 MB/s eta 0:00:00


In [ ]:
import pyodbc 
import pandas as pd
cnxn_str = ("Driver={odbc driver 17 for sql server};"
            "Server=DESKTOP-GVA3L0V\SQLEXPRESS;"
            "Database=prosain200411100082;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()	
cursor.execute("SELECT * FROM iris_arff") 
row = cursor.fetchall() 
cursor.close()
data=[]
for i in row:
    data.append([i[0],i[1],i[2]])
df_sqlServer = pd.DataFrame(data, columns=['id','petalwidth','class'])
df_sqlServer

In [ ]:
!pip install psycopg2

### Mengambil data dari database postgreSQL elephant

In [ ]:
import psycopg2
import pandas as pd

# Buat koneksi ke PostgreSQL.
conn = psycopg2.connect(database="rhwoswwx", user="rhwoswwx", password="2LqEviJ16iU8KE6PWVyjG0rdi_ZBgbHO", host="tiny.db.elephantsql.com")


# Buat cursor untuk menjalankan query SQL.
cur = conn.cursor()

# Jalankan query SQL.
cur.execute("SELECT ROW_NUMBER() OVER () as id, sepalwidth FROM iris_sepalwidth;")

# Ambil hasil query.
hasil_query = cur.fetchall()

df_postEle = pd.DataFrame(hasil_query, columns=["id", "sepalwidth"])

print(df_postEle)

# Tutup cursor dan koneksi.
cur.close()
conn.close()



### Mengambil data dari database Postgres Local

In [ ]:
import psycopg2
import pandas as pd

# Buat koneksi ke PostgreSQL.
conn = psycopg2.connect(database="prosaindata0082", user="postgres", password="12345", host="localhost", port="5432")


# Buat cursor untuk menjalankan query SQL.
cur = conn.cursor()

# Jalankan query SQL.
cur.execute("SELECT ROW_NUMBER() OVER () as id, sepallength FROM iris_arff;")

# Ambil hasil query.
hasil_query = cur.fetchall()

df_postLocal = pd.DataFrame(hasil_query, columns=["id", "sepallength"])

print(df_postLocal)

# Tutup cursor dan koneksi.
cur.close()
conn.close()

In [ ]:
!pip install mysqlclient
!pip install mysql-connector-python
!pip install pymysql

### Mengambil data dari database localhost

In [ ]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="prosaindata20041110082"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM iris")

myresult = mycursor.fetchall()
mycursor.close()
df_mysql = pd.DataFrame(myresult, columns=['id','petallength'])
# df_mysql.drop(df_mysql.columns[[0]],axis=1,inplace=True)
df_mysql

### Menggabungkan Empat Sumber data

In [ ]:
# merge df
result = pd.concat([df_postLocal, df_postEle,df_mysql, df_sqlServer], axis=1)
# result = pd.merge(df_postLocal,df_sqlServer, how="inner", on=["id", "id"])
result

### Menghapus Kolom ID

In [ ]:
result.drop(result.columns[[0,2,4,6]],axis=1,inplace=True)
result

In [ ]:
X = result.values[:, :4]
Y = result.values[:, 4]
print(X)

In [ ]:
X.shape

In [ ]:
Y.shape

### Model PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [ ]:
pca.fit(X)

In [ ]:
pca.components_

In [ ]:
Z = pca.transform(X)

In [ ]:
Z.shape

### Membagi Data Training dan Data Test
data traning sebesar 80% data test sebesar 20%

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print (" DecisionTree Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)